# Quantum Version of Fixed Strike Lookback Put Option

Note that more in-depth explanations please see the lookback call notebook.

In [1]:
import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
from tqdm import tqdm

from qiskit import QuantumCircuit
from qiskit.algorithms import IterativeAmplitudeEstimation, EstimationProblem
from qiskit.circuit.library import LinearAmplitudeFunction
from qiskit_aer.primitives import Sampler
from qiskit_finance.circuit.library import LogNormalDistribution

In [2]:
# Construct Uncertainty model helper function

def get_uncertainty_model(T, num_uncertainty_qubits=3, S=2.0, vol=0.4, r=0.0):
    # resulting parameters for log-normal distribution
    mu = (r - 0.5 * vol**2) * T + np.log(S)
    sigma = vol * np.sqrt(T)
    mean = np.exp(mu + sigma**2 / 2)
    variance = (np.exp(sigma**2) - 1) * np.exp(2 * mu + sigma**2)
    stddev = np.sqrt(variance)

    # lowest and highest value considered for the spot price; in between, an equidistant discretization is considered.
    low = np.maximum(0, mean - 3 * stddev)
    high = mean + 3 * stddev

    # construct A operator for QAE for the payoff function by
    # composing the uncertainty model and the objective
    uncertainty_model = LogNormalDistribution(
        num_uncertainty_qubits, mu=mu, sigma=sigma**2, bounds=(low, high)
    )
    return uncertainty_model, low, high

In [10]:
# Run Amplitude Estimation helper

def get_ae_results(uncertainty_model, low, high, num_uncertainty_qubits=3, strike_price=2.1, rescaling_factor=0.25, shots=100):
    # setup piecewise linear objective fcuntion
    breakpoints = [low, strike_price]
    slopes = [-1, 0]
    offsets = [strike_price - low, 0]
    f_min = 0
    f_max = strike_price - low
    european_put_objective = LinearAmplitudeFunction(
        num_uncertainty_qubits,
        slopes,offsets,
        domain=(low, high),
        image=(f_min, f_max),
        breakpoints=breakpoints,
        rescaling_factor=rescaling_factor,
    )

    # construct A operator for QAE for the payoff function by
    # composing the uncertainty model and the objective
    european_put = european_put_objective.compose(uncertainty_model, front=True)
    
    # set target precision and confidence level
    epsilon = 0.01
    alpha = 0.05

    problem = EstimationProblem(
        state_preparation=european_put,
        objective_qubits=[num_uncertainty_qubits],
        post_processing=european_put_objective.post_processing,
    )
    # construct amplitude estimation
    ae = IterativeAmplitudeEstimation(
        epsilon_target=epsilon, alpha=alpha, sampler=Sampler(run_options={"shots": shots})
    )
    result = ae.estimate(problem)
    expected_payoff = result.estimation_processed
    conf_int = np.array(result.confidence_interval_processed)
    return expected_payoff, conf_int

In [12]:
# Note that maximum payoff here occurs when the minimum spot price is achieved

# Run over the time steps, get maximum payoff
# Let's say 40 days, w/ time step = 1 day
T = 40 / 365
dt = 1 / 365
t = dt
results = []
conf_list = []
for i in range(40):
    um, low, high = get_uncertainty_model(t)
    res, conf = get_ae_results(um, low, high)
    results.append(res)
    conf_list.append(conf)
    t += dt
results = np.array(results)
print(f"Lookback Option Expected Payoff: {np.amax(results)}")
print(f"Payoff 95% CI w/ eps 0.01: {conf_list[np.argmax(results)]}")
print(f"Lookback Option Day of Max Payoff: {np.argmax(results)+1}")

Lookback Option Expected Payoff: 0.16617461196111832
Payoff 95% CI w/ eps 0.01: [0.16131107 0.17103815]
Lookback Option Day of Max Payoff: 40


In [13]:
# Now experiment with having say two consecutive days (e.g. a weekend/holiday) with a different uncertainty model
# To detect it with a lookback put, let's set days 20 & 21 to have 90% volatility

# Run over the time steps, get maximum payoff
# Let's say 40 days, w/ time step = 1 day
T = 40 / 365
dt = 1 / 365
t = dt
results = []
conf_list = []
for i in range(40):
    if i == 20 or i == 21:
        um, low, high = get_uncertainty_model(t, vol=0.9)
    else:
        um, low, high = get_uncertainty_model(t)
    res, conf = get_ae_results(um, low, high)
    results.append(res)
    conf_list.append(conf)
    t += dt
results = np.array(results)
print(f"Lookback Option Expected Payoff: {np.amax(results)}")
print(f"Payoff 95% CI w/ eps 0.01: {conf_list[np.argmax(results)]}")
print(f"Lookback Option Day of Max Payoff: {np.argmax(results)+1}")

Lookback Option Expected Payoff: 0.24512701882480292
Payoff 95% CI w/ eps 0.01: [0.2368079  0.25344614]
Lookback Option Day of Max Payoff: 22
